# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [2]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 
                                  'State', 'Metro', 'CountyName', 
                                  'SizeRank'], var_name='time')
    
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('zillow_data.csv')
df.head(1)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600


In [5]:
melt_df = melt_data(df)
melt_df.set_index('time', inplace=True)
melt_df.drop(['RegionID', 'SizeRank'], inplace=True, axis=1)
melt_df.tail()

,RegionName,City,State,Metro,CountyName,value
time,,,,,,
2018-04-01,1338,Ashfield,MA,Greenfield Town,Franklin,209300.0
2018-04-01,3293,Woodstock,NH,Claremont,Grafton,225800.0
2018-04-01,40404,Berea,KY,Richmond,Madison,133400.0
2018-04-01,81225,Mount Crested Butte,CO,NaN,Gunnison,664400.0
2018-04-01,89155,Mesquite,NV,Las Vegas,Clark,357200.0


# Step 2: Data Preprocessing

In [6]:
# # Scale the values
# scaler = MinMaxScaler()
# melt_df[['value']] = scaler.fit_transform(melt_df[['value']])

In [7]:
# sort by zip code and date
sorted_melt = melt_df.sort_values(['RegionName', 'time'])
sorted_melt.head()

,RegionName,City,State,Metro,CountyName,value
time,,,,,,
1996-04-01,1001,Agawam,MA,Springfield,Hampden,0.005274
1996-05-01,1001,Agawam,MA,Springfield,Hampden,0.005258
1996-06-01,1001,Agawam,MA,Springfield,Hampden,0.005248
1996-07-01,1001,Agawam,MA,Springfield,Hampden,0.005232
1996-08-01,1001,Agawam,MA,Springfield,Hampden,0.005222


In [8]:
# create a new data df containing data after 2009
new_df = sorted_melt[sorted_melt.index >= '2009-04-01']
new_df.head()

,RegionName,City,State,Metro,CountyName,value
time,,,,,,
2009-04-01,1001,Agawam,MA,Springfield,Hampden,0.009749
2009-05-01,1001,Agawam,MA,Springfield,Hampden,0.009713
2009-06-01,1001,Agawam,MA,Springfield,Hampden,0.009667
2009-07-01,1001,Agawam,MA,Springfield,Hampden,0.009630
2009-08-01,1001,Agawam,MA,Springfield,Hampden,0.009625


In [10]:
# first_obs = new_df[new_df.index == '2009-04-01']
# first_obs.head()

# last_obs = new_df[new_df.index == '2018-04-01']
# last_obs.head()

series09 = df['2009-04']
series18 = df['2018-04']
zipcode = df['RegionName']
diff_df = pd.concat([zipcode, series09, series18], axis=1)

diff_df['diff'] = (diff_df['2018-04'] - diff_df['2009-04'])/diff_df['2009-04']*100
diff_df.describe()
# %matplotlib inline
# diff_df.hist('diff',bins=20, density=True)
# diff_df.quantile([.1, .2, 3, .4, 5, 6, 7, 8, 9, 10], axis = 0) 
series09

0         803600.0
1         200700.0
2         245900.0
3        1037900.0
4         120300.0
5         123200.0
6         338500.0
7         667900.0
8         125900.0
9        2625300.0
10        536100.0
11        206600.0
12        651300.0
13        514100.0
14        141000.0
15        376000.0
16        336800.0
17        152500.0
18        171600.0
19        361200.0
20       6550900.0
21       5908200.0
22        189700.0
23        193000.0
24        153300.0
25        177500.0
26         85800.0
27        199700.0
28        124900.0
29        363100.0
           ...    
14693     182600.0
14694     425900.0
14695     162800.0
14696     229800.0
14697     240100.0
14698     214400.0
14699     214200.0
14700      65500.0
14701     187900.0
14702     170300.0
14703     601900.0
14704     647100.0
14705      69300.0
14706          NaN
14707          NaN
14708          NaN
14709     141200.0
14710     187000.0
14711    1524400.0
14712     158000.0
14713     119200.0
14714     23

In [ ]:
IQR = [diff_df.diff.quantile(0.1), diff_df.diff.quantile(0.2), diff_df.diff.quantile(0.3),
       diff_df.diff.quantile(0.4), diff_df.diff.quantile(0.5), diff_df.diff.quantile(0.6),
       diff_df.diff.quantile(0.7), diff_df.diff.quantile(0.8), diff_df.diff.quantile(0.9)]

# Step 3: Reshape from Wide to Long Format

# Step 4: EDA and Visualization
Perform basic EDA as you would building any model, then create at least 3 visualizations.  At least one of the visualizations should have time on the x-axis.  Axes should be labeled and any text should be legible.

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 5: ARIMA Modeling
Be sure to validate your model.  Make a forecast for the future and describe the trend.

# Step 6: Interpreting Results
Give a detailed interpretation of results including the relationship between the inputs and output of the model.